In [1]:
import numpy as np
import pandas as pd
#import time dependencies
from datetime import datetime
import time
from datetime import date, timedelta
import datetime
#import statistics dependencies
from scipy.stats import linregress
#import matplotlib dependencies
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.lines as mlines
# Import the API key.
from config import EIA_key
import eia
# Import the requests library.
import requests

In [2]:
# DUCInventory='duc-data.csv'
# DUCInventory_df=pd.read_csv(DUCInventory)
# DUCInventory_df=pd.read_csv(DUCInventory, header=[1])
# DUCInventory_df=pd.read_csv(DUCInventory, header=[2])
# # DUCInventory_df.set_index( , inplace=True)
# DUCInventory_df.columns = ['new_column_1_name','Anadarko_Drilled','Anadarko_Completed','Anadarko_DUC','Unnamed: 4','Appalachia_Drilled','Appalachia_Completed','Appalachia_DUC','Unnamed: 8','Bakken_Drilled','Bakken_Completed','Bakken_DUC','Unnamed: 12','Eagle Ford_Drilled','Eagle Ford_Completed', 'Eagle Ford_DUC', 'Unnamed: 16','Haynesville_Drilled','Haynesville_Completed','Haynesville_DUC', 'Unnamed: 20', 'Niobrara_Drilled','Niobrara_Completed', 'Niobrara_DUC','Unnamed: 24','Permian_Drilled','Permian_Completed', 'Permian_DUC','Unnamed: 28', 'DPR_Drilled','DPR_Completed','DPR_DUC']                                                                        
# # DUCInventory_df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
# DUCInventory_df

In [7]:
DUCInventory='duc-data.csv'
DUCInventory_df=pd.read_csv(DUCInventory, header=[1]) #header=[0,1]
DUCInventory_df=pd.read_csv(DUCInventory, header=[2])
# DUCInventory_df=DUCInventory_df.columns.droplevel(5)
DUCInventory_df.columns = ['Date','Anadarko_Drilled','Anadarko_Completed','Anadarko_DUC','Unnamed: 4','Appalachia_Drilled','Appalachia_Completed','Appalachia_DUC','Unnamed: 8','Bakken_Drilled','Bakken_Completed','Bakken_DUC','Unnamed: 12','Eagle Ford_Drilled','Eagle Ford_Completed', 'Eagle Ford_DUC', 'Unnamed: 16','Haynesville_Drilled','Haynesville_Completed','Haynesville_DUC', 'Unnamed: 20', 'Niobrara_Drilled','Niobrara_Completed', 'Niobrara_DUC','Unnamed: 24','Permian_Drilled','Permian_Completed', 'Permian_DUC','Unnamed: 28', 'DPR_Drilled','DPR_Completed','DPR_DUC']




# DUCInventory_df.rename(columns={'Anadarko':'Anadarko_Drilled'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 2':'Anadarko_Completed'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 3':'Anadarko_DUC'}, inplace=True)

# DUCInventory_df.rename(columns={'Appalachia':'Appalachia_Drilled'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 6':'Appalachia_Completed'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 7':'Appalachia_DUC'}, inplace=True)

# DUCInventory_df.rename(columns={'Bakken':'Bakken_Drilled'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 10':'Bakken_Completed'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 11':'Bakken_DUC'}, inplace=True)

# DUCInventory_df.rename(columns={'Eagle Ford':'Eagle Ford_Drilled'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 14':'Eagle Ford_Completed'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 15':'Eagle Ford_DUC'}, inplace=True)

# DUCInventory_df.rename(columns={'Haynesville':'Haynesville_Drilled'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 18':'Haynesville_Completed'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 19':'Haynesville_DUC'}, inplace=True)

# DUCInventory_df.rename(columns={'Niobrara':'Niobrara_Drilled'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 22':'Niobrara_Completed'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 23':'Niobrara_DUC'}, inplace=True)

# DUCInventory_df.rename(columns={'Permian':'Permian_Drilled'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 26':'Permian_Completed'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 27':'Permian_DUC'}, inplace=True)

# DUCInventory_df.rename(columns={'DPR Regions':'DPR_Drilled'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 30':'DPR_Completed'}, inplace=True)
# DUCInventory_df.rename(columns={'Unnamed: 31':'DPR_DUC'}, inplace=True)

del DUCInventory_df['Unnamed: 4']
del DUCInventory_df['Unnamed: 8']
del DUCInventory_df['Unnamed: 12']
del DUCInventory_df['Unnamed: 16']
del DUCInventory_df['Unnamed: 20']
del DUCInventory_df['Unnamed: 24']
del DUCInventory_df['Unnamed: 28']

DUCInventory_df=DUCInventory_df.iloc[2:81, ]
# DUCInventory_df['Date']=pd.to_datetime(DUCInventory_df['Date'],format='Y%m', errors='coerce')

# crude_df['Date'] = pd.to_datetime(crude_df['Date'], errors='coerce'
DUCInventory_df.tail(5)

,Date,Anadarko_Drilled,Anadarko_Completed,Anadarko_DUC,Appalachia_Drilled,Appalachia_Completed,Appalachia_DUC,Bakken_Drilled,Bakken_Completed,Bakken_DUC,...,Haynesville_DUC,Niobrara_Drilled,Niobrara_Completed,Niobrara_DUC,Permian_Drilled,Permian_Completed,Permian_DUC,DPR_Drilled,DPR_Completed,DPR_DUC
76,Mar-20,58,105,756,84,100,605,95,103,876,...,290,144,141,488,462,442,"3,404","1,085","1,100","7,657"
77,Apr-20,30,58,728,82,87,600,69,59,886,...,291,79,81,486,332,309,"3,427",745,730,"7,672"
78,May-20,13,29,712,61,76,585,33,30,889,...,292,35,40,481,200,195,"3,432",416,466,"7,622"
79,Jun-20,10,17,705,59,70,574,20,13,896,...,293,23,21,483,149,101,"3,480",324,291,"7,655"
80,Jul-20,9,15,699,57,63,568,19,13,902,...,294,16,15,484,138,98,"3,520",292,262,"7,685"


In [11]:
#access the first column
# data.iloc[:, <column_number>]
DUCInventory_df.iloc[:,0].dtypes
DUCInventory_df.head(5)

,Date,Anadarko_Drilled,Anadarko_Completed,Anadarko_DUC,Appalachia_Drilled,Appalachia_Completed,Appalachia_DUC,Bakken_Drilled,Bakken_Completed,Bakken_DUC,...,Haynesville_DUC,Niobrara_Drilled,Niobrara_Completed,Niobrara_DUC,Permian_Drilled,Permian_Completed,Permian_DUC,DPR_Drilled,DPR_Completed,DPR_DUC
2,Jan-14,230,215,313,140,108,"1,269",178,143,605,...,124,163,144,477,581,590,650,"1,654","1,569","4,429"
3,Feb-14,205,197,321,165,128,"1,306",210,158,657,...,136,172,126,523,583,519,714,"1,762","1,480","4,711"
4,Mar-14,216,238,299,121,178,"1,249",224,218,663,...,133,169,186,506,609,624,699,"1,720","1,874","4,557"
5,Apr-14,238,235,302,197,169,"1,277",217,207,673,...,123,194,194,506,657,632,724,"1,948","1,826","4,679"
6,May-14,210,232,280,184,207,"1,254",226,208,691,...,117,206,175,537,653,628,749,"1,877","1,836","4,720"


In [14]:
DUCInventory_df['Date'] = pd.to_datetime(DUCInventory_df["Date"]).dt.strftime('%m-%Y')
DUCInventory_df



# df['Month'] = pd.to_datetime(df["Month"].apply(lambda x: datetime.strptime(x,'%b-%y'))).dt.strftime('%m-%Y')

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-01-14 00:00:00